In [1]:
import jaydebeapi

In [2]:
import numpy
import pandas

<h3>環境設置</h3>

In [3]:
stock_ID = 'NQF'
data_table = 'StockInfo'

In [4]:
field_names = [
    'StockID', 'StockDate',
    'StockOpen', 'StockHigh', 'StockLow', 'StockClose',
    'StockAdjClose', 'StockVol'
]

<h3>開啟資料庫</h3>

In [5]:
dbConnection =  jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:D:\Projects\DB\data\H2",
    ["SA", ""],
    "D:/Java/h2/bin/h2-1.4.200.jar")

dbCursor = dbConnection.cursor()

<h3>資料統計-按月小計</h3>
<h3>最大/最小值</h3>

In [6]:
_sql = [
    "SELECT LEFT(STOCKDATE, 7) AS YM,", 
    "MIN(StockOpen) AS MinStockOpen, MAX(StockOpen) AS MaxStockOpen,",
    "AVG(StockOpen) AS AvgStockOpen, (MAX(StockOpen) + MIN(StockOpen)) * 0.5 AS ScaleStockOpen",
    "FROM STOCKINFO",
    "WHERE StockID='%s'" % stock_ID, 
    "GROUP BY LEFT(StockDate, 7)",
    "HAVING AVG(StockOpen) > (MAX(StockOpen) + MIN(StockOpen)) * 0.5" #HAVING：針對答案進行篩選；WHERE：針對資料進行篩選
]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall() 

rows = { 
    'YM' : [],
    'MIN_OPEN' : [],
    'MAX_OPEN' : [],
    'AVG_OPEN' : [],
    'SCALE_OPEN' : []
} 

for row in resultSet:
    rows['YM'].append(row[0])
    rows['MIN_OPEN'].append(row[1])
    rows['MAX_OPEN'].append(row[2])
    rows['AVG_OPEN'].append(row[3])
    rows['SCALE_OPEN'].append(row[4])
    
results = pandas.DataFrame(data=rows).set_index(['YM'])

results

,MIN_OPEN,MAX_OPEN,AVG_OPEN,SCALE_OPEN
YM,,,,
2017-01,4876.75,5148.25,5040.500000,5012.500
2017-02,5129.50,5352.50,5247.973684,5241.000
2017-06,5648.25,5880.25,5775.897727,5764.250
2017-07,5591.25,5961.00,5816.144737,5776.125
2017-08,5769.00,5936.25,5871.782609,5852.625
2017-09,5878.75,6001.75,5958.600000,5940.250
2017-12,6255.50,6535.00,6407.100000,6395.250
2018-01,6405.00,7030.00,6774.940476,6717.500
2018-02,6328.00,7003.50,6713.597222,6665.750


<h3>不使用Having指令，而是用Python語法達成</h3>

In [7]:
_sql = [
    "SELECT STOCKDATE, LEFT(STOCKDATE, 7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE StockID='%s'" % stock_ID,
]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows = {
    'STOCKDATE' : [],
    'YM' : [],
    'STOCKOPEN' : []
}

for row in resultSet:
    rows['STOCKDATE'].append(row[0])
    rows['YM'].append(row[1])
    rows['STOCKOPEN'].append(row[2])
    
results = pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min', 'mean', 'max'])
results.columns = [['MIN_OPEN', 'AVG_OPEN', 'MAX_OPEN']]
results['SCALE_OPEN'] = 0.5 * numpy.add(results['MAX_OPEN'].values, results['MIN_OPEN'].values)

In [8]:
condition = [ ]

for i in range(results.shape[0]): #shape[0]：資料有多少筆
    row = results.iloc[i]
    condition.append(row['SCALE_OPEN'] > row['AVG_OPEN'])
    
results[condition]

,MIN_OPEN,AVG_OPEN,MAX_OPEN,SCALE_OPEN
YM,,,,
2017-03,5334.25,5381.304348,5440.75,5387.500
2017-04,5352.75,5442.052632,5584.50,5468.625
2017-05,5575.50,5676.806818,5793.00,5684.250
2017-10,5984.00,6080.909091,6221.75,6102.875
2017-11,6230.00,6319.250000,6423.50,6326.750
2018-04,6405.75,6627.690476,6855.25,6630.500
2018-08,7284.25,7446.445652,7672.75,7478.500
2018-09,7433.00,7541.289474,7670.00,7551.500
2019-03,7022.25,7259.190476,7526.75,7274.500


In [9]:
_year = 2018

_sql = [
    "SELECT LEFT(STOCKDATE, 7) AS YM,",
    "MIN(StockOpen) AS MinStockOpen, MAX(StockOpen) AS MaxStockOpen,",
    "AVG(StockOpen) AS AvgStockOpen, (MAX(StockOpen) + MIN(StockOpen)) * 0.5 AS ScaleStockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE, 4)=%d AND StockID='%s'" % (_year, stock_ID),
    "GROUP BY LEFT(StockDate, 7)",
    "HAVING AVG(StockOpen) > (MAX(StockOpen) + MIN(StockOpen)) * 0.5"
]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows = {
    'YM' : [],
    'MIN_OPEN' : [],
    'AVG_OPEN' : [],
    'MAX_OPEN' : []
} 

for row in resultSet:
    rows['YM'].append(row[0])
    rows['MIN_OPEN'].append(row[1])
    rows['AVG_OPEN'].append(row[2])
    rows['MAX_OPEN'].append(row[3])
    
results = pandas.DataFrame(data=rows).set_index(['YM'])

results.head()

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2018-01,6405.00,7030.00,6774.940476
2018-02,6328.00,7003.50,6713.597222
2018-03,6490.75,7136.75,6868.773810
2018-05,6606.00,6989.50,6868.863636
2018-06,6982.50,7315.25,7169.369048


In [10]:
def Calc_Open(year, stock_id):
    _sql = [
        "SELECT STOCKDATE, LEFT(STOCKDATE, 7) AS YM, StockOpen",
        "FROM STOCKINFO",
        "WHERE LEFT(STOCKDATE, 4)=%d AND StockID='%s'" % (year, stock_id),
    ]
    
    dbCursor.execute(' '.join(_sql))
    resultSet = dbCursor.fetchall() 
    
    rows = { 
        'STOCKDATE' : [],
        'YM' : [],
        'STOCKOPEN' : []
    } 

    for row in resultSet:
        rows['STOCKDATE'].append(row[0])
        rows['YM'].append(row[1])
        rows['STOCKOPEN'].append(row[2])

    results = pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min', 'mean', 'max'])
    results.columns = [['MIN_OPEN', 'AVG_OPEN', 'MAX_OPEN']]
    
    return results

In [11]:
df_2019 = Calc_Open(year=2019, stock_id='NQF')
df_2019

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2019-01,6168.75,6595.333333,6844.25
2019-02,6863.75,7009.960526,7122.00
2019-03,7022.25,7259.190476,7526.75
2019-04,7430.00,7663.333333,7855.25
2019-05,7214.75,7517.511364,7848.75
2019-06,7008.75,7489.975000,7753.00
2019-07,7781.50,7915.190476,8017.50
2019-08,7349.00,7623.875000,7834.00
2019-09,7614.25,7816.912500,7916.25


In [12]:
df_all = None

for _year in [2017, 2018, 2019]:
    _df = Calc_Open(year=_year, stock_id=stock_ID)
    if df_all is None:
        df_all = _df
    else:
        df_all =pandas.concat([df_all, _df])
        
df_all

,MIN_OPEN,AVG_OPEN,MAX_OPEN
YM,,,
2017-01,4876.75,5040.500000,5148.25
2017-02,5129.50,5247.973684,5352.50
2017-03,5334.25,5381.304348,5440.75
2017-04,5352.75,5442.052632,5584.50
2017-05,5575.50,5676.806818,5793.00
2017-06,5648.25,5775.897727,5880.25
2017-07,5591.25,5816.144737,5961.00
2017-08,5769.00,5871.782609,5936.25
2017-09,5878.75,5958.600000,6001.75


In [13]:
df_all.shape[0]

36

In [14]:
dbCursor.close()
dbConnection.close()